<center>
<img src="../../img/ml_theme.png">
# Майнор "Интеллектуальный анализ данных" 
# Курс "Введение в анализ данных"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватели ФКН НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="http://www.microsoft.com/en-us/openness/default.aspx#Ms-RL">Ms-RL</a>. Можно использовать в любых целях, но с обязательным упоминанием автора курса и аффилиации.

# Семинар 13. Продвинутые методы классификации и регрессии

## Часть 3.  kNN с метрикой, определенной пользователем

In [6]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
%pylab inline
import numpy as np
from math import sqrt
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV
import sys
sys.path.append('../../scripts/')
from load_car_insurance_with_region import load_train_and_test
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import StratifiedKFold

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [3]:
# read data
X_train, y, X_test = map(np.array, 
                         load_train_and_test("../../data/car_insurance_train.csv",
                                         "../../data/car_insurance_test.csv"))

In [5]:
def my_dist_metric(region_weight, brand_weight):
    def dist(x,y):
        return sqrt(region_weight * (x[2] - y[2]) ** 2 + 
                    brand_weight * (x[0] - y[0]) ** 2 +
                    (1 - region_weight - brand_weight) * (x[1] - y[1]) ** 2)
    return dist

In [7]:
%%time
results = []

params = [(neighbors, reg_weight, brand_weight, 
           1 - reg_weight - brand_weight)
          for reg_weight in np.arange(0, 1, 0.05)
          for brand_weight in np.arange(0, 1 - reg_weight, 0.05)
          for neighbors in range(1,12)]

for (n_neighbors, reg_weight, brand_weight, comp_weight) in params:
        scores = []
        dist_func = my_dist_metric(reg_weight, brand_weight)
        
        knn = KNeighborsClassifier(n_neighbors=n_neighbors,
                                   metric='pyfunc', 
                                   func=dist_func)
        
        folds = StratifiedKFold(y, n_folds=3, 
                                random_state=42)
        for train_index, test_index in folds:
            x_train, x_test = X_train[train_index], \
                                  X_train[test_index]
            y_train, y_test = y[train_index], y[test_index]
            knn.fit(x_train, y_train)
            scores.append(roc_auc_score(knn.predict(x_test), y_test))
        results.append(np.mean(scores))
            

CPU times: user 12min 39s, sys: 6.47 s, total: 12min 46s
Wall time: 15min


In [8]:
max_auc = max(results)
max_auc, params[np.argmax(results)]

(0.69411868694027545, (11, 0.0, 0.050000000000000003, 0.94999999999999996))

In [9]:
knn = KNeighborsClassifier(n_neighbors=11,
                           metric='pyfunc', 
                           func=my_dist_metric(.0, .05))
knn.fit(X_train, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='pyfunc',
           metric_params={'func': <function dist at 0x109211848>},
           n_neighbors=11, p=2, weights='uniform')

In [10]:
predicted_labels = knn.predict(X_test)

# # turn predictions into data frame and save as csv file
predicted_df = pd.DataFrame(predicted_labels,
                            index = np.arange(1, X_test.shape[0] + 1),
                            columns=["too_much"])
predicted_df.to_csv("../../output/knn_car_insurance_custom_metric.csv", 
                    index_label="id")

# that's for those who know the answers :)
expected_labels_df = pd.read_csv("../../data/car_insurance_test_labels.csv",
                                 header=0, index_col=0)
expected_labels = expected_labels_df['too_much']
print(roc_auc_score(predicted_labels, expected_labels))

0.761104441777
